### Actions
N(0), E(1), S(2), W(3)

### States
N(0), NE(1), E(2), SE(3), S(4), SW(5), W(6), NW(7)


### Rewards
1. +10 Eats Food
2. -0.1 Moves

In [1]:
from matplotlib import pyplot as plt
import matplotlib.animation
import random
import numpy as np
import json

In [198]:
WIDTH = 25
HEIGHT = 25

In [199]:
def get_possible_actions(current_pos):
   # N(0), E(1), S(2), W(3)

    actions =[]
    if current_pos[0] > 0:
        actions.append(0)
    if current_pos[0] < HEIGHT-1:
        actions.append(2)
    if current_pos[1] < WIDTH-1:
        actions.append(1)
    if current_pos[1] > 0:
        actions.append(3)
   
    
    return actions

def get_next_pos(current_pos, action):
    if(action == 0):
        return [current_pos[0] - 1,current_pos[1]] 
    elif (action == 2):
        return [current_pos[0] + 1,current_pos[1]] 
    elif (action == 1):
        return [current_pos[0],current_pos[1]+1]
    else:
        return [current_pos[0],current_pos[1]-1]


def get_state(current_pos, food_pos):
# N(0), NE(1), E(2), SE(3), S(4), SW(5), W(6), NW(7)
    
    if current_pos[0] > food_pos[0] and current_pos[1] == food_pos[1]:
        return 0
    elif current_pos[0] > food_pos[0] and current_pos[1] < food_pos[1]:
        return 1
    elif current_pos[0] == food_pos[0] and current_pos[1] < food_pos[1]:
        return 2
    elif current_pos[0] < food_pos[0] and current_pos[1] < food_pos[1]:
        return 3
    elif current_pos[0] < food_pos[0] and current_pos[1] == food_pos[1]:
        return 4
    elif current_pos[0] < food_pos[0] and current_pos[1] > food_pos[1]:
        return 5
    elif current_pos[0] == food_pos[0] and current_pos[1] > food_pos[1]:
        return 6
    elif current_pos[0] > food_pos[0] and current_pos[1] > food_pos[1]:
        return 7
    else:
        return None

def game_complete(score):
    return score > 50 or score < -10

def set_food_pos(current_pos):
    food_pos = [random.randint(0, HEIGHT-1), random.randint(0, WIDTH-1)]
    while food_pos == current_pos:
        food_pos = [random.randint(0, HEIGHT-1), random.randint(0, WIDTH-1)]
    return food_pos

def evaluate_move(current_pos, food_pos):
    if current_pos == food_pos:
        return 10
    else: 
        return -1


In [200]:
print(get_possible_actions([2,2]))

[0, 2, 1, 3]


In [201]:
def generate_env_q_matrix(height, width):
    env = [[-0.1 for x in range(width)] for y in range(height)]
    
    # N(0), NE(1),E(2), SE(3), S(4), SW(5), W(6), NW(7)
    q_matrix = [[0,0,0,0] for x in range(8)]
    
    
    return env, q_matrix

In [202]:
env, q_matrix = generate_env_q_matrix(HEIGHT, WIDTH)
print(q_matrix)


[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]


In [205]:
discount = 0.9
learning_rate = 0.1

for _ in range(100000):
    score = 0

    current_pos = [random.randint(0, HEIGHT-1), random.randint(0, WIDTH-1)]
    food_pos = set_food_pos(current_pos)
    

    while(not game_complete(score)):

        state = get_state(current_pos, food_pos)
        if state == None:
            food_pos = set_food_pos(current_pos)
            state = get_state(current_pos, food_pos)


        possible_actions = get_possible_actions(current_pos)
        action = random.choice(possible_actions)
        next_pos = get_next_pos(current_pos, action)
        reward = evaluate_move(next_pos, food_pos)



        q_matrix[state][action] = q_matrix[state][action]  + learning_rate * (reward + discount *  max(q_matrix[state]) - q_matrix[state][action])
        
       
        current_pos = next_pos
        score -= 0.1


# values = []
directions = []

for row in q_matrix:
    # values.append(column[np.argmax(column)])
    directions.append(np.argmax(row))
        
# values = np.reshape(values,(4,4))
# directions = np.reshape(directions,(4,4))

print(json.dumps(q_matrix, indent = 2))
print(directions)



[
  [
    -7.443374691571338,
    -8.023705288874234,
    -8.044186177088209,
    -7.822799327272537
  ],
  [
    -9.999999999999908,
    -9.999999999999908,
    -9.999999999999908,
    -9.999999999999908
  ],
  [
    -6.304131995097807,
    -5.766626520915445,
    -6.178437584718446,
    -6.292362007933039
  ],
  [
    -9.999999999999908,
    -9.999999999999908,
    -9.999999999999908,
    -9.999999999999908
  ],
  [
    -6.194279230223108,
    -6.22019058708915,
    -5.9758797105697035,
    -6.2359898119877695
  ],
  [
    -9.999999999999908,
    -9.999999999999908,
    -9.999999999999908,
    -9.999999999999908
  ],
  [
    -5.033701492161919,
    -5.083241232385661,
    -5.0380534068278395,
    -3.6304540483940393
  ],
  [
    -9.999999999999908,
    -9.999999999999908,
    -9.999999999999908,
    -9.999999999999908
  ]
]
[0, 0, 1, 0, 2, 0, 3, 0]


In [207]:
env, _ = generate_env_q_matrix (HEIGHT ,WIDTH)

current_pos = [random.randint(0, HEIGHT-1), random.randint(0, WIDTH-1)]
env[current_pos[0]][current_pos[1]] = 50

food_pos = set_food_pos(current_pos)
env[food_pos[0]][food_pos[1]] = 100



fig, ax = plt.subplots()
l = ax.imshow(env)

def move():
    global current_pos
    global food_pos
    
    
    state = get_state(current_pos, food_pos)
    

    if state == None:
        env[food_pos[0]][food_pos[1]] = 0
        food_pos = set_food_pos(current_pos)
        env[food_pos[0]][food_pos[1]] = 100
        state = get_state(current_pos, food_pos)

    action = directions[state]
    print(state, action)

    env[current_pos[0]][current_pos[1]] = 0
    current_pos = get_next_pos(current_pos, action)
    env[current_pos[0]][current_pos[1]] = 50


def anim(i):
    l.set_data(env)
    move()


ani = matplotlib.animation.FuncAnimation(fig, anim, frames=25)


plt.close()
from IPython.display import HTML
HTML(ani.to_jshtml())




0 0
0 0
0 0
0 0
1 0
1 0
1 0
1 0
1 0
1 0
1 0
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
2 1
5 0
5 0
